## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [43]:
import os
from dotenv import load_dotenv
load_dotenv() 

groq_api_key = os.getenv("GROQ_API_KEY")


if groq_api_key is None:
    groq_api_key = "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" 
    print("Using manual API key")
else:
    print("Using API key from environment variables")

print(f"API key loaded: {'Yes' if groq_api_key else 'No'}")

Using API key from environment variables
API key loaded: Yes


In [44]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D89A7A290>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D89A797B0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [45]:
from langchain_core.messages import HumanMessage

# You need to replace this with your actual valid Groq API key
# Get your API key from: https://console.groq.com/keys
valid_groq_api_key = "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"

# Create a new model instance with valid API key
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=valid_groq_api_key)

model.invoke([HumanMessage(content="Hi , My name is harsh and I am a  AI Engineer")])

AIMessage(content="Hi Harsh, it's nice to meet you!\n\nThat's awesome that you're an AI Engineer. It's such a fascinating and rapidly evolving field. \n\nWhat kind of work are you involved in?  Are you working on anything particularly interesting or challenging at the moment?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 22, 'total_tokens': 85, 'completion_time': 0.114545455, 'prompt_time': 0.002129884, 'queue_time': 0.242929845, 'total_time': 0.116675339}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7ee2ecd8-d284-4429-b6d9-76f236fd7418-0', usage_metadata={'input_tokens': 22, 'output_tokens': 63, 'total_tokens': 85})

In [46]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat. 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 97, 'total_tokens': 141, 'completion_time': 0.08, 'prompt_time': 0.004297039, 'queue_time': 0.245341121, 'total_time': 0.084297039}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--5e3e45cf-e2df-4dbb-b7f3-b35e0b53d45c-0', usage_metadata={'input_tokens': 97, 'output_tokens': 44, 'total_tokens': 141})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [47]:
!pip install langchain_community

In [48]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [49]:
config={"configurable":{"session_id":"chat1"}}

In [50]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is harsh and I am a Chief AI Engineer")],
    config=config
)

In [51]:
response.content

"Hello Harsh, it's nice to meet you!  \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?  \n\nI'm eager to learn more about your work in the field of AI.\n"

In [52]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Harsh, as you told me at the beginning of our conversation. 😊 \n\nIs there anything else I can help you with today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 90, 'total_tokens': 123, 'completion_time': 0.06, 'prompt_time': 0.004084459, 'queue_time': 0.24234257799999998, 'total_time': 0.064084459}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--9471c2c6-6a37-4308-8983-6c0a6261b8d3-0', usage_metadata={'input_tokens': 90, 'output_tokens': 33, 'total_tokens': 123})

In [53]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [54]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! 👋 \n\nIt's nice to meet you.  \n\nIs there anything I can help you with today?  😄  \n\n"

In [55]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  😊  \n\nI remember that you told me earlier.  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [56]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [57]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is harsh")]})

AIMessage(content="Hi Harsh! \n\nIt's nice to meet you.  \n\nHow can I help you today? 😊  \n\nAsk me anything! \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 31, 'total_tokens': 65, 'completion_time': 0.061818182, 'prompt_time': 0.002369444, 'queue_time': 0.245222303, 'total_time': 0.064187626}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--933ae839-f6f0-464f-80ad-b422001fe984-0', usage_metadata={'input_tokens': 31, 'output_tokens': 34, 'total_tokens': 65})

In [58]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [59]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is harsh")],
    config=config
)

response

AIMessage(content="Hi Harsh, it's nice to meet you! 👋\n\nI'm happy to help answer any questions you have. Just ask away! 😊  What can I do for you today? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_time': 0.078181818, 'prompt_time': 0.002351704, 'queue_time': 0.245762026, 'total_time': 0.080533522}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--bf385be7-c69a-474a-aaca-1b8db70cbe45-0', usage_metadata={'input_tokens': 31, 'output_tokens': 43, 'total_tokens': 74})

In [60]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Harsh!  I remember that from when you introduced yourself. 😊  \n\nIs there anything else I can help you with?\n'

In [61]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [62]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is har")],"language":"Hindi"})
response.content

'नमस्ते हर,  मैं आपकी मदद करने के लिए यहाँ हूँ। आप किस बारे में पूछना चाहते हैं? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [63]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [64]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! 😊 \n\nमुझे ख़ुशी है कि आप मुझसे बात करना चाहते हैं।  आप किस बारे में जानना चाहते हैं?  \n\nमैं आपकी मदद करने के लिए तैयार हूँ!\n\n'

In [65]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [66]:
response.content

'आपका नाम कृष्ण है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [67]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [68]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor. \n\nWhat's your favorite ice cream? 😊🍦\n"

In [69]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

"You asked what 2 + 2 equals.  😊  \n\n\n\nIs there another one you'd like to try?\n"

In [70]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [71]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [72]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. Every interaction we have is fresh and new.\n\nIf you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what you need.  \n\n"

## Deploying Your Chatbot Online

Now that we have a working chatbot, let's explore different ways to deploy it online so others can use it:

### Deployment Options:
1. **Streamlit** - Quick web app deployment
2. **Flask/FastAPI** - RESTful API service
3. **Gradio** - ML model interface

### Hosting Platforms:
- **Streamlit Cloud** (Free)
- **Heroku** (Free tier available)
- **Railway** (Free tier available)
- **Render** (Free tier available)
- **Vercel** (Free for hobby projects)

Let's start with the easiest option - Streamlit!

### Option 1: Streamlit Web App

Streamlit is the easiest way to create a web interface for your chatbot. First, let's install streamlit:

In [73]:
!pip install streamlit

Now let's create a Streamlit app file. Run the cell below to create the app:

In [78]:
# Create a Streamlit app file
streamlit_app_code = '''
import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Set page config
st.set_page_config(page_title="AI Chatbot", page_icon="🤖")

# Title
st.title("🤖 AI Chatbot")
st.write("Chat with our AI assistant powered by Groq!")

# Sidebar for API key
with st.sidebar:
    st.header("Configuration")
    groq_api_key = st.text_input("Enter Groq API Key:", type="password")
    if not groq_api_key:
        st.warning("Please enter your Groq API key to continue.")
        st.info("Get your API key from: https://console.groq.com/keys")

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

if "store" not in st.session_state:
    st.session_state.store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in st.session_state.store:
        st.session_state.store[session_id] = ChatMessageHistory()
    return st.session_state.store[session_id]

if groq_api_key:
    # Initialize the model
    try:
        model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
        
        # Create prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
            MessagesPlaceholder(variable_name="messages")
        ])
        
        chain = prompt | model
        
        # Wrap with message history
        with_message_history = RunnableWithMessageHistory(chain, get_session_history)
        
        # Display chat messages
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.write(message["content"])
        
        # Chat input
        if prompt_input := st.chat_input("Type your message here..."):
            # Add user message to chat history
            st.session_state.messages.append({"role": "user", "content": prompt_input})
            with st.chat_message("user"):
                st.write(prompt_input)
            
            # Generate response
            with st.chat_message("assistant"):
                with st.spinner("Thinking..."):
                    config = {"configurable": {"session_id": "streamlit_session"}}
                    response = with_message_history.invoke(
                        [HumanMessage(content=prompt_input)],
                        config=config
                    )
                    st.write(response.content)
            
            # Add assistant response to chat history
            st.session_state.messages.append({"role": "assistant", "content": response.content})
    
    except Exception as e:
        st.error(f"Error initializing chatbot: {str(e)}")
        st.info("Please check your API key and try again.")

# Instructions
with st.expander("ℹ️ How to use"):
    st.write("""
    1. Enter your Groq API key in the sidebar
    2. Start chatting with the AI assistant
    3. The bot will remember your conversation history
    4. You can ask questions, have conversations, or request help with various topics
    """)
'''

# Write the Streamlit app to a file
with open('streamlit_chatbot.py', 'w') as f:
    f.write(streamlit_app_code)

print("✅ Streamlit app created: streamlit_chatbot.py")
print("\nTo run the app locally:")
print("streamlit run streamlit_chatbot.py")

✅ Streamlit app created: streamlit_chatbot.py

To run the app locally:
streamlit run streamlit_chatbot.py


### Option 2: Flask API

Create a REST API that can be integrated into any application:

In [75]:
!pip install flask flask-cors

In [76]:
# Create a Flask API
flask_app_code = '''
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Global variables
store = {}
model = None
with_message_history = None

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

def initialize_model(api_key):
    global model, with_message_history
    try:
        model = ChatGroq(model="Gemma2-9b-It", groq_api_key=api_key)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
            MessagesPlaceholder(variable_name="messages")
        ])
        
        chain = prompt | model
        with_message_history = RunnableWithMessageHistory(chain, get_session_history)
        return True
    except Exception as e:
        print(f"Error initializing model: {e}")
        return False

@app.route('/', methods=['GET'])
def home():
    return jsonify({
        "message": "AI Chatbot API is running!",
        "endpoints": {
            "POST /chat": "Send a message to the chatbot",
            "GET /health": "Check API health"
        }
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "healthy", "model_initialized": model is not None})

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.json
        
        if not data or 'message' not in data:
            return jsonify({"error": "Message is required"}), 400
        
        api_key = data.get('api_key') or os.getenv('GROQ_API_KEY')
        if not api_key:
            return jsonify({"error": "API key is required"}), 401
        
        # Initialize model if not already done
        if model is None:
            if not initialize_model(api_key):
                return jsonify({"error": "Failed to initialize model"}), 500
        
        message = data['message']
        session_id = data.get('session_id', 'default_session')
        
        # Generate response
        config = {"configurable": {"session_id": session_id}}
        response = with_message_history.invoke(
            [HumanMessage(content=message)],
            config=config
        )
        
        return jsonify({
            "response": response.content,
            "session_id": session_id
        })
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    # Try to initialize with environment variable
    api_key = os.getenv('GROQ_API_KEY')
    if api_key:
        initialize_model(api_key)
    
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

# Write the Flask app to a file
with open('flask_chatbot.py', 'w') as f:
    f.write(flask_app_code)

print("✅ Flask API created: flask_chatbot.py")
print("\nTo run the API locally:")
print("python flask_chatbot.py")
print("\nAPI will be available at: http://localhost:5000")

✅ Flask API created: flask_chatbot.py

To run the API locally:
python flask_chatbot.py

API will be available at: http://localhost:5000


### Deployment Files

Let's create the necessary files for deployment:

In [77]:
# Create requirements.txt
requirements = '''
streamlit>=1.28.0
langchain-groq>=0.1.0
langchain-core>=0.2.0
langchain-community>=0.2.0
python-dotenv>=1.0.0
flask>=2.3.0
flask-cors>=4.0.0
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements.strip())

# Create a simple HTML interface for the Flask API
html_interface = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Chatbot</title>
    <style>
        body { font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }
        .chat-container { border: 1px solid #ddd; height: 400px; overflow-y: scroll; padding: 20px; margin-bottom: 20px; }
        .message { margin-bottom: 10px; padding: 10px; border-radius: 5px; }
        .user { background-color: #e3f2fd; text-align: right; }
        .assistant { background-color: #f5f5f5; }
        .input-container { display: flex; gap: 10px; }
        input[type="text"] { flex: 1; padding: 10px; border: 1px solid #ddd; border-radius: 5px; }
        button { padding: 10px 20px; background-color: #2196F3; color: white; border: none; border-radius: 5px; cursor: pointer; }
        button:hover { background-color: #1976D2; }
        .api-key-input { margin-bottom: 20px; }
    </style>
</head>
<body>
    <h1>🤖 AI Chatbot</h1>
    
    <div class="api-key-input">
        <input type="password" id="apiKey" placeholder="Enter your Groq API key" style="width: 100%; margin-bottom: 10px;">
    </div>
    
    <div class="chat-container" id="chatContainer"></div>
    
    <div class="input-container">
        <input type="text" id="messageInput" placeholder="Type your message..." onkeypress="handleKeyPress(event)">
        <button onclick="sendMessage()">Send</button>
    </div>

    <script>
        const API_URL = 'http://localhost:5000/chat';
        
        function handleKeyPress(event) {
            if (event.key === 'Enter') {
                sendMessage();
            }
        }
        
        function addMessage(content, isUser) {
            const chatContainer = document.getElementById('chatContainer');
            const messageDiv = document.createElement('div');
            messageDiv.className = `message ${isUser ? 'user' : 'assistant'}`;
            messageDiv.textContent = content;
            chatContainer.appendChild(messageDiv);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }
        
        async function sendMessage() {
            const messageInput = document.getElementById('messageInput');
            const apiKeyInput = document.getElementById('apiKey');
            const message = messageInput.value.trim();
            const apiKey = apiKeyInput.value.trim();
            
            if (!message) return;
            if (!apiKey) {
                alert('Please enter your Groq API key');
                return;
            }
            
            addMessage(message, true);
            messageInput.value = '';
            
            try {
                const response = await fetch(API_URL, {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({
                        message: message,
                        api_key: apiKey,
                        session_id: 'web_session'
                    })
                });
                
                const data = await response.json();
                
                if (response.ok) {
                    addMessage(data.response, false);
                } else {
                    addMessage(`Error: ${data.error}`, false);
                }
            } catch (error) {
                addMessage(`Error: ${error.message}`, false);
            }
        }
    </script>
</body>
</html>
'''

with open('chatbot_interface.html', 'w') as f:
    f.write(html_interface)

print("✅ Created deployment files:")
print("  - requirements.txt")
print("  - chatbot_interface.html")

✅ Created deployment files:
  - requirements.txt
  - chatbot_interface.html


### Deployment Instructions

#### 1. Streamlit Cloud (Easiest - Free)
1. Push your code to GitHub repository
2. Go to [share.streamlit.io](https://share.streamlit.io)
3. Connect your GitHub account
4. Deploy `streamlit_chatbot.py`
5. Add your `GROQ_API_KEY` in the secrets

#### 2. Railway (Free Tier)
1. Go to [railway.app](https://railway.app)
2. Create new project from GitHub
3. Add environment variable: `GROQ_API_KEY`
4. Deploy automatically

#### 3. Render (Free Tier) - Detailed Guide

**Render** is an excellent platform for deploying web applications with a generous free tier. Here's a complete step-by-step guide:

##### Prerequisites:
- GitHub account with your chatbot code
- Render account (free at [render.com](https://render.com))
- Your Groq API key

##### Step-by-Step Deployment:

**Step 1: Prepare Your Repository**
1. Ensure all files are in your GitHub repository:
   - `streamlit_chatbot.py` (main app file)
   - `requirements.txt` (dependencies)
   - Optional: `flask_chatbot.py` and `chatbot_interface.html`

**Step 2: Create a Render Account**
1. Go to [render.com](https://render.com)
2. Sign up with your GitHub account (recommended)
3. Authorize Render to access your repositories

**Step 3: Create a New Web Service**
1. Click **"New +"** → **"Web Service"**
2. Connect your GitHub repository
3. Select the repository containing your chatbot
4. Click **"Connect"**

**Step 4: Configure Your Service**
```
Name: my-ai-chatbot (or any name you prefer)
Region: Choose closest to your users
Branch: main (or your default branch)
Runtime: Python 3
Build Command: pip install -r requirements.txt
Start Command: streamlit run streamlit_chatbot.py --server.port=$PORT --server.address=0.0.0.0
```

**Step 5: Set Environment Variables**
1. Scroll down to **"Environment Variables"**
2. Click **"Add Environment Variable"**
3. Add:
   - **Key:** `GROQ_API_KEY`
   - **Value:** Your actual Groq API key
4. Optionally add:
   - **Key:** `PORT`
   - **Value:** `8501` (Streamlit default port)

**Step 6: Choose Plan**
1. Select **"Free"** plan (limited but good for testing)
2. Free tier includes:
   - 750 hours/month (about 31 days)
   - Automatic sleep after 15 minutes of inactivity
   - Custom domain support
   - HTTPS included

**Step 7: Deploy**
1. Click **"Create Web Service"**
2. Render will automatically:
   - Clone your repository
   - Install dependencies
   - Start your application
3. Watch the build logs for any errors

**Step 8: Access Your App**
1. Once deployed, you'll get a URL like:
   `https://my-ai-chatbot-abc123.onrender.com`
2. Your chatbot is now live and accessible worldwide!

##### Render-Specific Configuration:

**For Streamlit Apps:**
- Build Command: `pip install -r requirements.txt`
- Start Command: `streamlit run streamlit_chatbot.py --server.port=$PORT --server.address=0.0.0.0 --server.headless=true --server.enableCORS=false --server.enableXsrfProtection=false`

**For Flask Apps:**
- Build Command: `pip install -r requirements.txt`
- Start Command: `python flask_chatbot.py`

##### Troubleshooting Common Issues:

**1. Build Failures:**
- Check `requirements.txt` for correct package names
- Ensure all dependencies are listed
- Check build logs for specific error messages

**2. App Won't Start:**
- Verify start command is correct
- Check for syntax errors in your code
- Ensure environment variables are set correctly

**3. API Key Issues:**
- Double-check your Groq API key is valid
- Ensure environment variable name matches your code
- Check API key has proper permissions

**4. Port Issues:**
- Always use `$PORT` environment variable in start command
- Don't hardcode port numbers
- For Streamlit: `--server.port=$PORT`

##### Free Tier Limitations:
- **Sleep Mode:** App sleeps after 15 minutes of inactivity
- **Cold Starts:** May take 30-60 seconds to wake up
- **Resource Limits:** 512MB RAM, shared CPU
- **Build Time:** Limited monthly build minutes

##### Upgrading to Paid Plans:
- **Starter ($7/month):** No sleep, faster builds, more resources
- **Standard ($25/month):** Dedicated resources, faster performance

##### Advanced Features:**
- **Custom Domains:** Point your own domain to the app
- **Auto-Deploy:** Automatic deployment on git push
- **Environment Groups:** Manage variables across services
- **Health Checks:** Monitor app status
- **Logs:** Real-time application logs

##### Security Best Practices:**
- Never commit API keys to your repository
- Use environment variables for all secrets
- Enable HTTPS (included by default)
- Consider adding authentication for production use

#### 4. Local Testing
```bash
# For Streamlit
streamlit run streamlit_chatbot.py

# For Flask API
python flask_chatbot.py
# Then open chatbot_interface.html in your browser
```

#### 5. Environment Variables
Make sure to set these environment variables on your hosting platform:
- `GROQ_API_KEY`: Your Groq API key

#### 6. Security Notes
- Never commit API keys to your repository
- Use environment variables for sensitive data
- Consider implementing rate limiting for production use
- Add authentication if needed